Zbiór danych z którymi pracowałem zamieściłem na GH. 
W celu stworzenia odpowiednich zasobów poczyniłem następne kroki na platformie Azure: 

1. Zalogowanie się na AML wraz ze stworzeniem zasobu

2. Pobranie odpowiedniego zbioru danych 

3. Stworzenie zasobów obliczeniowych Compute Instances i Compute Clusters

4. Załadowanie danych jako dataset

5. Stworzenie pliku .ipynb jako Notebooka

Inspirowałem się plikiem z githuba który był w opisie laboratorium. 


Poniżej znajduje się dokładny flow operacji do zbadania działania modelu. Z uwagi na potwarzający się wielokrotnie błąd: 

``ModuleNotFoundError: No module named 'azureml.automl.runtime._ml_engine.featurizer_suggestion'``
podczas określania najlepszego modelu, nie udało mi się wykonać całego zaplanowanego laboratorium. 

W celu naprawy usterki wykonałem następujące kroki:

1. Upgrade paczek za pomocą menadżera ``pip`` - udało mi się zrobić update SDK z 1.19 do wersji 1.21. 

2. Restart wszystkich zasobów związanych z serwisem. 

3. Stworzenie nowego zbiou danych i notebooka 

4. Upewnienie się czy wersja SDK której używa notebook jest aktualna. 


Poniżej znajduje się dokładny flow operacji. Zasoby utworzone są na moim profilu na Azure. 

In [3]:
import azureml.core
from azureml.core import Workspace, Dataset
ws = Workspace.from_config()

Po stworzeniu workspace i załadowaniu podstawowych paczek przechodzę do załadowania danych z przesłanego wcześniej pliku. 

In [5]:
aml_dataset = ws.datasets['covid']
# Use Pandas DataFrame just to sneak peak some data and schema
full_df = aml_dataset.to_pandas_dataframe()
# .to_pandas_dataframe().dropna()
full_df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,None,597.029,9.59,None,None,37.746,0.5,64.83,0.498
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,None,597.029,9.59,None,None,37.746,0.5,64.83,0.498
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,None,597.029,9.59,None,None,37.746,0.5,64.83,0.498
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,None,597.029,9.59,None,None,37.746,0.5,64.83,0.498
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,None,597.029,9.59,None,None,37.746,0.5,64.83,0.498


In [6]:
full_df.describe()


,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
count,6.145000e+04,61443.000000,60484.000000,5.291300e+04,52912.000000,60484.000000,61097.000000,61090.000000,60136.000000,52573.000000,...,59046.000000,58362.000000,58711.000000,59082.000000,59683.000000,60386.000000,29888.000000,54816.000000,61395.000000,59347.000000
mean,3.202574e+05,3202.012369,3195.428938,1.022976e+04,80.488056,69.064089,5553.759014,59.798426,59.592909,137.536739,...,30.620154,8.838208,5.603401,19179.028279,256.331253,7.770147,51.116083,3.041553,73.139299,0.715704
std,3.152440e+06,28435.125924,28092.605460,8.195278e+04,600.016639,538.278972,11279.237872,165.517508,137.326083,254.602912,...,9.120537,6.254834,4.272178,19665.274056,118.125481,3.895157,31.773684,2.470955,7.549240,0.153772
min,1.000000e+00,-46076.000000,-1121.714000,1.000000e+00,-1918.000000,-232.143000,0.001000,-2153.437000,-276.825000,0.001000,...,15.100000,1.144000,0.526000,661.240000,79.370000,0.990000,1.188000,0.100000,53.280000,0.354000
25%,5.090000e+02,1.000000,4.429000,2.800000e+01,0.000000,0.000000,106.391000,0.069000,0.823000,4.777000,...,22.200000,3.466000,2.043000,4541.795000,164.905000,5.290000,20.859000,1.300000,67.440000,0.590000
50%,5.316000e+03,43.000000,52.928500,1.630000e+02,1.000000,0.714000,823.673000,4.912000,6.621000,24.359000,...,29.900000,6.614000,3.871000,13111.214000,241.219000,7.110000,49.839000,2.400000,74.620000,0.747000
75%,5.428400e+04,492.000000,524.143000,1.485000e+03,11.000000,8.714000,5147.281000,44.831500,49.919250,127.701000,...,39.100000,14.312000,9.167000,27216.445000,325.721000,10.080000,83.241000,4.000000,78.570000,0.843000
max,9.919440e+07,856352.000000,738108.429000,2.129409e+06,17817.000000,14082.571000,123587.653000,8652.658000,2648.773000,1915.257000,...,48.200000,27.049000,18.493000,116935.600000,724.417000,30.530000,98.999000,13.800000,86.750000,0.953000


Tworzę zbiory testowe i treningowe dla modelu.

In [7]:
train_dataset, test_dataset = aml_dataset.random_split(0.8, seed=1)

train_dataset_df = train_dataset.to_pandas_dataframe()
test_dataset_df = test_dataset.to_pandas_dataframe()

print(train_dataset_df.describe())

        total_cases      new_cases  new_cases_smoothed  total_deaths  \
count  4.929400e+04   49288.000000        48548.000000  4.250300e+04   
mean   3.182305e+05    3149.882385         3155.875921  1.014245e+04   
std    3.155530e+06   28067.280612        27955.619269  8.190531e+04   
min    1.000000e+00  -46076.000000        -1121.714000  1.000000e+00   
25%    5.090000e+02       1.000000            4.429000  2.800000e+01   
50%    5.360500e+03      43.000000           52.857000  1.630000e+02   
75%    5.403600e+04     490.000000          522.606750  1.468000e+03   
max    9.919440e+07  856352.000000       738108.429000  2.129409e+06   

         new_deaths  new_deaths_smoothed  total_cases_per_million  \
count  42503.000000         48548.000000             49003.000000   
mean      78.813519            67.916387              5554.034022   
std      591.910739           534.740354             11306.429131   
min    -1918.000000          -232.143000                 0.001000   
25%   

Korzystam z mojego cluster-ml i sprawdzam czy zasób istnieje na platformie Azure. 

In [8]:
amlcompute_cluster_name = "cluster-ml"

found = False
cts = ws.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
     found = True
     print('Found existing training cluster.')
     aml_remote_compute = cts[amlcompute_cluster_name]
    
if not found:
     print('Creating not found')
    
print('Checking cluster status...')
aml_remote_compute.wait_for_completion(show_output = True)

Found existing training cluster.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
from azureml.train import automl

automl.utilities.get_primary_metrics('regression')

['spearman_correlation',
 'r2_score',
 'normalized_root_mean_squared_error',
 'normalized_mean_absolute_error']

Tworzę podstawową konfiguracje dla eksperymentu. Wykorzystam regresje w celu zbadania ilości zakażeń. 

In [10]:
import logging
import os

from azureml.train.automl import AutoMLConfig

project_folder = './'
os.makedirs(project_folder, exist_ok=True)

automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='regression',
                             primary_metric='r2_score',
                             experiment_timeout_minutes=15,                            
                             training_data=train_dataset,
                             label_column_name="total_cases",
                             n_cross_validations=5,                                                 
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             path=project_folder
                             )

Uruchamiam eksperyment, oznaczając go odpowiednią datą w celu dobrej dokumentacji na przyszłość. 

In [11]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "regress-automl-remote-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=ws, name=experiment_name)

import time
start_time = time.time()
            
run = experiment.submit(automl_config, show_output=True)


print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))

regress-automl-remote-01-27-2021-13
Running on remote.
No run_configuration provided, running on cluster-ml with default configuration
Running on remote compute: cluster-ml
Parent Run ID: AutoML_3861647f-1d9f-4767-a5ae-5dac869598b7

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
          

Tutaj suzkam najlepszego modelu. Dostaję błąd opisany wcześnej. 

In [15]:
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

ModuleNotFoundError: No module named 'azureml.automl.runtime._ml_engine.featurizer_suggestion'

In [ ]:
import pandas as pd

if 'price' in test_dataset_df.columns:
    y_test_df = test_dataset_df.pop('total_cases')

x_test_df = test_dataset_df

Predykcja na zbiorze testowym, któru uprzednio stworzyłem. 

In [ ]:
y_predictions = fitted_model.predict(x_test_df)

print('10 predictions: ')
print(y_predictions[:10])

Wyliczenie wartości r2_score. 

In [ ]:
from sklearn.metrics import r2_score

print('Result: r2 score')
r2_score(y_test_df, y_predictions)